## Name : Tirth Hihoriya
## Roll no : 18bce244
## Prac- 5 : CIFAR10_Transfer_learning

In [2]:
import tensorflow
from tensorflow import keras
import cv2
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from keras.layers import Dense, MaxPool2D, Conv2D, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.datasets import cifar10

In [3]:
(X_train, y_train), (X_test,y_test) = cifar10.load_data()
print("X_train -> ",X_train.shape)
print("y_train -> ",y_train.shape)
print("X_test  -> ",X_test.shape)
print("y_test  -> ",y_test.shape)

X_train ->  (50000, 32, 32, 3)
y_train ->  (50000, 1)
X_test  ->  (10000, 32, 32, 3)
y_test  ->  (10000, 1)


In [4]:
X_train  =X_train[:10000]
y_train  =X_train[:10000]
X_test   =X_test[:2000]
y_test   =y_test[:2000]
print("X_train -> ",X_train.shape)
print("y_train -> ",y_train.shape)
print("X_test  -> ",X_test.shape)
print("y_test  -> ",y_test.shape)

X_train ->  (10000, 32, 32, 3)
y_train ->  (10000, 32, 32, 3)
X_test  ->  (2000, 32, 32, 3)
y_test  ->  (2000, 1)


### One hot encoding

In [ ]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
print(y_train_one_hot.shape)
print(y_test_one_hot.shape)

In [ ]:
total_classes = y_train_one_hot.shape[1]
total_classes

## Resize for Xception

In [ ]:
X_train_ = np.array([cv2.resize(X_train[i], dsize=(72,72), interpolation=cv2.INTER_CUBIC) for i in range(len(X_train))]).astype('float16')
X_test_ = np.array([cv2.resize(X_test[i], dsize=(72,72), interpolation=cv2.INTER_CUBIC) for i in range(len(X_test))]).astype('float16')
print(X_train_.shape)
print(X_test_.shape)

In [ ]:
X_train_ = preprocess_input(X_train_)
X_test_ = preprocess_input(X_test_)
print(X_train_.shape)
print(X_test_.shape)

In [ ]:
model_pretrained = Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(72,72,3), pooling=None)

In [ ]:
model_pretrained.summary()

In [ ]:
X_train_ = model_pretrained.predict(X_train_)
X_test_ = model_pretrained.predict(X_test_)
print(X_train_.shape)
print(X_test_.shape)

## Classifier on top

In [ ]:
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=(7,7,2048)))
model.add(Dropout(0.3))
model.add(Dense(10, activation= 'softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_, y_train_one_hot, validation_data=(X_test_, y_test_one_hot), batch_size=32, epochs=3, verbose=1)

In [ ]:
scores = model.evaluate(x=X_test_, y=y_test_one_hot, verbose=1)
print(model.metrics_names)
print("Test Loss :", scores[0])
print("Baseline Error:", round((100-scores[1]*100),2),'%')